In [1]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest
import pickle
import pymongo

Δεδομένων όλων των ομιλιών, πρέπει να ανιχνεύσουμε ομοιότητες ανά ζεύγη μεταξύ των μελών του κοινοβουλίου. Συγκεκριμένα, πρέπει να βρούμε έναν τρόπο να εξαγάγουμε ένα διάνυσμα χαρακτηριστικών για κάθε μέλος και στη συνέχεια να εκτελέσουμε ομοιότητες ανά ζεύγη για να μπορέσουμε να ανιχνεύσουμε τα top-k ζεύγη με τον υψηλότερο βαθμό ομοιότητας (όπου k είναι μια παράμετρος).

In [2]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#mongo_client.drop_database("GreekParliamentProceedings")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]
print(type(database))

<class 'pymongo.collection.Collection'>


In [3]:
with open('results/member_keywords.pickle', 'rb') as handle:
    keywords_by_member = pickle.load(handle)
#print(list(keywords_by_member['τσιπρας παυλου αλεξιος'].keys()))
member_names = keywords_by_member.keys()
keywords = [list(keywords_by_member[member].keys()) for member in member_names]

### Διάνυσμα χαρακτηριστικών για κάθε μέλος

In [ ]:
#CREATE A VECTOR FOR EACH MEMBER IN A PANDAS DATAFRAME
member_names = keywords_by_member.keys()
keywords = [list(keywords_by_member[member].keys()) for member in member_names]
genders = []
political_parties = []
member_roles = []
regions = []
governments = []
def get_attributes_by_member(members, collection):
        for member in members:
                print(member)
                pipeline = [{'$match' : {'member_name':member}},
                            {'$facet': {
                                'groupByRoles':[{'$group':{'_id':'$roles'}}],
                                'groupByGender':[{'$group':{'_id':'$member_gender'}}],
                                'groupByGovernment':[{'$group':{'_id':'$government'}}],
                                'groupByParty':[{'$group':{'_id':'$political_party'}}],
                                'groupByRegion':[{'$group':{'_id':'$member_region'}}]
                            }
                                        }
                        ]
                r = list(collection.aggregate(pipeline))
                genders.append(r[0]['groupByGender'][0]['_id'])
                regions.append([r[0]['groupByRegion'][i]['_id'] for i in range(len(r[0]['groupByRegion']))])
                member_roles.append([r[0]['groupByRoles'][i]['_id'] for i in range(len(r[0]['groupByRoles']))])
                governments.append([r[0]['groupByGovernment'][i]['_id'] for i in range(len(r[0]['groupByGovernment']))])
                political_parties.append([r[0]['groupByParty'][i]['_id'] for i in range(len(r[0]['groupByParty']))])
                
        return genders, regions, political_parties, member_roles, governments

In [ ]:
genders, regions, political_parties, member_roles, governments = get_attributes_by_member(member_names, database)

In [ ]:
print(len(member_names))
print(len(keywords))
print(len(genders))
print(len(regions))
print(len(member_roles))
print(len(governments))
print(len(political_parties))

In [ ]:
#CREATE PANDAS DATAFRAME
data = {'Name': member_names,
        'Gender': genders,
        'Region(s)':regions,
        'Role(s)':member_roles,
        'Government(s)':governments,
        'Political party':political_parties,
        'Keywords':keywords}
df = pd.DataFrame(data)
df.head(4)

### Ομοιότητες ανά ζεύγη: MinHash + LSH

In [4]:
from lsh import LSH

LSH = LSH(keywords)
candidates_pairs = LSH.MinHashLSH(num_hashes=256, num_bands=32, num_columns=4)
print(len(candidates_pairs))

34024


In [5]:
accepted = {}
for pair in candidates_pairs:
    sim = LSH.jaccard_similarity(keywords[pair[0]], keywords[pair[1]])
    if(sim>0.7):
        if(sim not in accepted.keys()):
            accepted[sim] = [pair]
        else:
            accepted[sim].append(pair)

In [7]:
print(accepted.keys())
print(accepted[0.8])

dict_keys([0.7142857142857143, 0.9230769230769231, 0.8571428571428571, 0.7857142857142857, 0.7272727272727273, 1.0, 0.75, 0.7333333333333333, 0.8, 0.7692307692307693])
[[817, 437]]


In [8]:
print(keywords[817])
print(keywords[437])

['νομοσχεδ', 'αρθρ', 'νε δημοκρατ ', 'ευρ']
['πασοκ', 'ευρ', 'νε δημοκρατ ', 'αρθρ', 'νομοσχεδ']
